# Evaluate AI agents of Foundry Agent Services

## Objective


This sample demonstrates how to evaluate an AI agent (Azure AI Agent Service) on these important aspects of your agentic workflow:

- Intent Resolution: Measures how well the agent identifies the user’s request, including how well it scopes the user’s intent, asks clarifying questions, and reminds end users of its scope of capabilities.

For AI agents outside of Azure AI Agent Service, you can still provide th agent data in the two formats (either simple data or agent messages) specified in the individual evaluator samples:
- [Intent resolution](https://aka.ms/intentresolution-sample)
<!--
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample)
- [Task adherence](https://aka.ms/taskadherence-sample)
- [Response Completeness](https://aka.ms/rescompleteness-sample)
-->


## Time 

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
Creating an agent using Azure AI agent service requires an Azure AI Foundry project and a deployed, supported model. See more details in [Create a new agent](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal).

For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

Important: Make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Prerequisite

Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```

### Setup Azure credentials and project 
1. use az cli to login to the tenant with your credential

<!-- initializing Project Client -->

In [1]:
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv(dotenv_path=".env", override=True)

from utils.fdyauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


In [ ]:
import os
import azure.ai.agents as agentslib
import azure.ai.projects as projectslib
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentEvaluationRequest,
    InputDataset,
    EvaluatorIds,
    EvaluatorConfiguration,
    AgentEvaluationSamplingConfiguration,
    AgentEvaluationRedactionConfiguration,
    Evaluation,
    DatasetVersion,
    FileDatasetVersion,
)
from azure.ai.agents.models import (
    FunctionTool,
    ToolSet,
    MessageRole,
)

# Import your custom functions to be used as Tools for the Agent
from utils.user_functions import user_functions

# Initialize project client with proper authentication
project_client = AIProjectClient(
    credential=credential,  # Use the credential from earlier setup
    endpoint=settings.project_endpoint
)
print("project_client api version:", project_client._config.api_version)
print(f"azure-ai-agents version: {agentslib.__version__}")
print(f"azure-ai-projects version: {projectslib.__version__}")

AGENT_NAME = "Seattle Tourist Agent"
AGENT_INSTRUCTIONS = """You are a helpful tourist assistant"""

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4)

project_client api version: 2025-05-15-preview
azure-ai-agents version: 1.1.0b3
azure-ai-projects version: 1.0.0b12


### Create an AI agent (Azure AI Agent Service)

In [3]:
found_agent = None
all_agents_list = project_client.agents.list_agents()
for a in all_agents_list:
    if a.name == AGENT_NAME:
        found_agent = a
        break

if found_agent:
    agent = project_client.agents.update_agent(
        agent_id=found_agent.id,
        model=settings.model_deployment_name,
        instructions=AGENT_INSTRUCTIONS,
        toolset=toolset,
    )
    project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4) 
    print(f"reusing agent > {agent.name} (id: {agent.id})")
else:
    agent = project_client.agents.create_agent(
        model=settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
        toolset=toolset,
    )
    print(f"Created agent '{AGENT_NAME}' with {len(functions._functions)} tools\nID: {agent.id}")

reusing agent > Seattle Tourist Agent (id: asst_rs0qLwEnvDVyoxWkp740d8S8)


## Conversation with Agent
Use below cells to have conversation with the agent
1. `Create a thread`
2. `Create Message`
3. `Execute`

### Create Thread - 1

In [4]:
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_CwuvWVkYl5PomvD7Kh9X9twE


### Create Message - 2

In [5]:
# Create a new user message and add it to the thread (state)
MESSAGE = "Can you email me weather info for Seattle ? My email is user@microsoft.com."
# MESSAGE = "Can you email me weather info for Seattle ?"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_Qjq9lnb0mDA9RK5sfUGiWPHR


### Execute - 3

In [6]:
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Sending email to user@microsoft.com...
Subject: Weather Information for Seattle
Body:
The current weather in Seattle is rainy with a temperature of 14°C.
Run finished with status: RunStatus.COMPLETED
Run ID: run_dakWQEiHPPWoECDmrCKN0KXh


### List Messages

In [7]:
for message in project_client.agents.messages.list(thread.id, order="asc"):
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Can you email me weather info for Seattle ? My email is user@microsoft.com.
----------------------------------------
Role: MessageRole.AGENT
Content: I have sent the weather information for Seattle to your email (user@microsoft.com). If you need any more information or assistance, feel free to ask!
----------------------------------------


# Evaluate

### Evaluation in the cloud

Reference:
* https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/cloud-evaluation
* https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/cloud-evaluation#prerequisite-set-up-steps-for-azure-ai-foundry-projects

For the Microsoft Entra ID, give MSI (Microsoft Identity) permissions for "Storage Blob Data Owner" through IAM to both
* `User, group, or service principal` "EntraID user" and
* `Managed Identity` by adding the Role its `Azure AI Foundry Project`
from the storage account IAM.
* And make sure to choose "Share to all project" while adding the storage account connection to the Azure AI Foundry Project v2
* Blob Storage need to have public network access, so that foundry project can upload the blob file

<!--
Adding additional `Azure AI Administrator Role` to the Microsoft EntraID User for the `Azure AI Foundry Resource`

* `Managed Identity` by adding the Role to both `Azure AI Foundry Resource` and its `Azure AI Foundry Project`
from the storage account IAM.
-->

### Prepare the data from agent response for evaluation

Reference:
* https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/agent-evaluate-sdk#evaluate-azure-ai-agents

In [9]:
from azure.ai.evaluation import AIAgentConverter
import json

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id

# Get a single agent run data for evaluation
single_agent_eval_input_data = converter.convert(thread_id=thread_id, run_id=run_id)

# make folder
dir_path = os.path.join(os.getcwd(), "data")
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# Save the agent run data to a JSONL file
# eval_file_name = f"single_agent_eval_{thread_id}_{run_id}.jsonl"
eval_file_name = f"singleagentrun.jsonl"
eval_file_path = os.path.join(os.getcwd(), "data", eval_file_name)

print(single_agent_eval_input_data)

Class AIAgentConverter: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FDPAgentDataRetriever: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AIAgentDataRetriever: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'query': [{'role': 'system', 'content': 'You are a helpful tourist assistant'}, {'createdAt': '2025-07-07T13:18:52Z', 'role': 'user', 'content': [{'type': 'text', 'text': 'Can you email me weather info for Seattle ? My email is user@microsoft.com.'}]}], 'response': [{'createdAt': '2025-07-07T13:18:54Z', 'run_id': 'run_dakWQEiHPPWoECDmrCKN0KXh', 'role': 'assistant', 'content': [{'type': 'tool_call', 'tool_call_id': 'call_l41AzZ6R0lp6XcW6l8DiefkQ', 'name': 'fetch_weather', 'arguments': {'location': 'Seattle'}}]}, {'createdAt': '2025-07-07T13:18:56Z', 'run_id': 'run_dakWQEiHPPWoECDmrCKN0KXh', 'tool_call_id': 'call_l41AzZ6R0lp6XcW6l8DiefkQ', 'role': 'tool', 'content': [{'type': 'tool_result', 'tool_result': {'weather': 'Rainy, 14°C'}}]}, {'createdAt': '2025-07-07T13:18:57Z', 'run_id': 'run_dakWQEiHPPWoECDmrCKN0KXh', 'role': 'assistant', 'content': [{'type': 'tool_call', 'tool_call_id': 'call_JSp5z9K7V9659si36p16Jsjx', 'name': 'send_email', 'arguments': {'recipient': 'user@microsoft.com', 

In [10]:
from utils.converter import extract_agent_data
eval_input = extract_agent_data(single_agent_eval_input_data, ground_truth="yes, I mailed you the weather info for Seattle")
eval_input

{'query': 'Can you email me weather info for Seattle ? My email is user@microsoft.com.',
 'ground_truth': 'yes, I mailed you the weather info for Seattle',
 'response': 'I have sent the weather information for Seattle to your email (user@microsoft.com). If you need any more information or assistance, feel free to ask!',
 'context': 'You are a helpful tourist assistant',
 'latency': 8.5,
 'response_length': 149}

In [11]:
# save the the whole object of single_agent_eval_input_data to the eval_file_path as jsonl entry
# how to append to existing file
with open(eval_file_path, "w") as f:
    f.write(json.dumps(eval_input))

### AI Foundry Cloud Evaluation

In [12]:
# generate a timestamp of now
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

In [13]:
print("Upload a single file and create a new Dataset to reference the file.")
dataset_name = f"singleagent-{timestamp}"
dataset_version = "1.0"

existing_datasets = project_client.datasets.list()
map = {}
for dataset in existing_datasets:
    # print(f"Dataset:{dataset}")
    print(f"Dataset: {dataset.name}, Version: {dataset.version}, ID: {dataset.id}")
    map[dataset.name] = dataset.id

if dataset_name in map:
    print(f"Dataset {dataset_name} already exists with ID: {map[dataset_name]}")
    project_client.datasets.delete(name=dataset_name, version=dataset_version)
    print(f"Deleted existing dataset {dataset_name} with version {dataset_version}")

# stacctaievalywuno with IAM role "storage blob data owner" for the Azure AI Foundry project and Entra ID principal 
dataset: DatasetVersion = project_client.datasets.upload_file(
    name=dataset_name,
    version=dataset_version,
    file_path=eval_file_path,
    connection_name="stacctaievalywuno"
)
print(dataset)

Upload a single file and create a new Dataset to reference the file.
Dataset: singleagent-20250707151622, Version: 1.0, ID: azureai://accounts/foundry-proj-yw-uno-resource/projects/foundry-proj-yw-uno/data/singleagent-20250707151622/versions/1.0
Dataset: singleagent-20250707151228, Version: 1.0, ID: azureai://accounts/foundry-proj-yw-uno-resource/projects/foundry-proj-yw-uno/data/singleagent-20250707151228/versions/1.0
Dataset: singleagent-20250707150913, Version: 1.0, ID: azureai://accounts/foundry-proj-yw-uno-resource/projects/foundry-proj-yw-uno/data/singleagent-20250707150913/versions/1.0
Dataset: singleagent-20250707150742, Version: 1.0, ID: azureai://accounts/foundry-proj-yw-uno-resource/projects/foundry-proj-yw-uno/data/singleagent-20250707150742/versions/1.0
Dataset: singleagent-20250707142357, Version: 1.0, ID: azureai://accounts/foundry-proj-yw-uno-resource/projects/foundry-proj-yw-uno/data/singleagent-20250707142357/versions/1.0
Dataset: singleagent-20250707135213, Version: 

#### Textual similarity evaluators

* https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-evaluators/textual-similarity-evaluators
* Config examples https://github.com/Azure-Samples/azureai-samples/blob/main/scenarios/evaluate/Supported_Evaluation_Metrics/AI_Judge_Evaluators_Quality/AI_Judge_Evaluators_Quality.ipynb

In [16]:
print("Create an evaluation task")
evaluation: Evaluation = Evaluation(
    display_name=f"Single Agent Eval {timestamp}",
    description="Evaluation for seattle tourist agent",
    # Sample Dataset Id : azureai://accounts/<account_name>/projects/<project_name>/data/<dataset_name>/versions/<version>
    data=InputDataset(id=dataset.id if dataset.id else ""),
    evaluators={
        # AI Quality Eval, AI Assisst 
        "relevance": EvaluatorConfiguration(
            id=EvaluatorIds.RELEVANCE.value,
            init_params={
                "deployment_name": settings.model_deployment_name,
            },
            data_mapping={
                "query": "${data.query}",
                "response": "${data.response}",
            },
        ),
        # "groundedness": EvaluatorConfiguration(
        #     id=EvaluatorIds.GROUNDEDNESS.value,
        #     init_params={
        #         "deployment_name": settings.model_deployment_name,
        #     },
        #     data_mapping={
        #         "response": "${target.response}",
        #         # "context": "${data.context}",
        #         "query": "${data.query}",
        #     },
        # ),
        # "fluency": EvaluatorConfiguration(
        #     id=EvaluatorIds.FLUENCY.value,
        #     init_params={
        #         "deployment_name": settings.model_deployment_name,
        #     },
        #     data_mapping={
        #         "response": "${target.response}",
        #         # "context": "${data.context}",
        #         "query": "${data.query}",
        #     },
        # ),
        # Risks and Safety: content filter
        "violence": EvaluatorConfiguration(
            id=EvaluatorIds.VIOLENCE.value,
            init_params={
                "azure_ai_project": settings.project_endpoint,
            },
        ),
        # "sexual": EvaluatorConfiguration(
        #     id=EvaluatorIds.SEXUAL.value,
        #     init_params={
        #         "azure_ai_project": settings.project_endpoint,
        #     },
        # ),
        # "self_harm": EvaluatorConfiguration(
        #     id=EvaluatorIds.SELF_HARM.value,
        #     init_params={
        #         "azure_ai_project": settings.project_endpoint,
        #     },
        # ),
        # text evaluators
        "bleu_score": EvaluatorConfiguration(
            id=EvaluatorIds.BLEU_SCORE.value,
            init_params={
                "threshold": 0.01,
            },
            data_mapping={
                "response": "${data.response}",
                "ground_truth": "${data.ground_truth}",
            },
        ),
        "f1_score": EvaluatorConfiguration(
            id=EvaluatorIds.F1_SCORE.value,
            init_params={
                "threshold": 0.2,
            },
            data_mapping={
                "response": "${data.response}",
                "ground_truth": "${data.ground_truth}",
            },
        ),
        "meteor_score": EvaluatorConfiguration(
            id=EvaluatorIds.METEOR_SCORE.value,
            init_params={
                "threshold": 0.4,
            },
            data_mapping={
                "response": "${data.response}",
                "ground_truth": "${data.ground_truth}",
            },
        ),
    },
)

# Use the model endpoint and API key as AI Evaluator to run the evaluation
evaluation_response: Evaluation = project_client.evaluations.create(
    evaluation,
    headers={
        "model-endpoint": settings.azure_openai_endpoint,
        "api-key": settings.azure_openai_api_key,
    },
)
print(evaluation_response)

print("Get evaluation")
get_evaluation_response: Evaluation = project_client.evaluations.get(evaluation_response.name)

print(get_evaluation_response)

print("List evaluations")
for evaluation in project_client.evaluations.list():
    print(f"{evaluation.display_name}:{evaluation.description}")

Create an evaluation task
{'data': {'id': 'azureai://accounts/foundry-proj-yw-uno-resource/projects/foundry-proj-yw-uno/data/singleagent-20250707152046/versions/1.0', 'type': 'Dataset'}, 'target': None, 'description': 'Evaluation for seattle tourist agent', 'evaluators': {'relevance': {'id': 'azureai://built-in/evaluators/relevance', 'initParams': {'deployment_name': 'gpt-4.1-mini'}, 'dataMapping': {'query': '${data.query}', 'response': '${data.response}'}}, 'violence': {'id': 'azureai://built-in/evaluators/violence', 'initParams': {'azure_ai_project': 'https://foundry-proj-yw-uno-resource.services.ai.azure.com/api/projects/foundry-proj-yw-uno'}, 'dataMapping': {}}, 'bleu_score': {'id': 'azureai://built-in/evaluators/bleu_score', 'initParams': {'threshold': 0.01}, 'dataMapping': {'response': '${data.response}', 'ground_truth': '${data.ground_truth}'}}, 'f1_score': {'id': 'azureai://built-in/evaluators/f1_score', 'initParams': {'threshold': 0.2}, 'dataMapping': {'response': '${data.resp

In [15]:
# List all evaluation runs in the project
print("List all evaluations in the project")
eval_list = project_client.evaluations.list()
for eval_item in eval_list:
    print(f"Evaluation ID: {eval_item.display_name}")

List all evaluations in the project
Evaluation ID: Single Agent Eval 20250707152046
Evaluation ID: Single Agent Eval 20250707151622
Evaluation ID: Single Agent Eval 20250707151228
Evaluation ID: Single Agent Eval 20250707150913
Evaluation ID: Single Agent Eval 20250707150742
Evaluation ID: Single Agent Eval 20250707142357
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707135213
Evaluation ID: Single Agent Eval 20250707132440
Evaluation ID: Single Agent Eval 20250707132440
Evaluation ID: Single Agent Eval 20250707132440
Evaluation ID: Single Agent Eval 20250707132440
Evaluation ID: Single Agent Eval 20250707132440
Evaluation ID: Single Agent Eval 20250707132440
Eval

### Foundry Project Evaluation Metric Dashboard

1. Login to `https://ai.azure.com/`
2. choose your Foundry Project
3. Open the `Evaluation` menu item
4. choose an evaluation run to see metric dashboard

![](imgs/cloud_eval_metric_dashboard.png)